In [11]:
import requests

import pandas as pd

from datetime import datetime, timedelta

from pprint import pprint

In [45]:
lang = input('¿En español (es) o en ingles (en)?')
category = input('¿Balance, demanda, generacion o intercambio?')
widget = input('¿Qué widget quieres?')

restaDia = float(input('¿Cuantos días atrás?:____________'))
input_año = int(input('¿Qué año?:_____________'))

In [43]:
headers = {
    "Accept": "application/json",
    "Content_Type": "application/json",
    "Host": "apidatos.ree.es"
}

In [46]:
def demanda_datos(lang, category, widget, query):
    
    now = datetime.now()
    ultima_fecha = (now - timedelta(days = restaDia)).replace(year = input_año).strftime('%Y-%m-%d')


    hoy = now.replace(year = input_año).strftime('%Y-%m-%d')

    query = f"start_date={ultima_fecha}T00:00&end_date={hoy}T23:59&time_trunc=day"



    endpoint = f"https://apidatos.ree.es/{lang}/datos/{category}/{widget}?{query}"
    response = requests.get(url = endpoint, headers= headers)
    data = response.json()

    datetime_lista = list()
    value_lista = list()
    percentage_lista = list()


    response = requests.get(endpoint)
    data = response.json()

    for value in data['included']:
        for content in value['attributes']['values']:
                porcentaje = content['percentage']
                valor = content['value']
                fecha = content ['datetime']


                datetime_lista.append(fecha)
                value_lista.append(valor)
                percentage_lista.append(porcentaje)


    df_demanda = pd.DataFrame()
    df_demanda['fecha'] = datetime_lista
    df_demanda['energia_consumida /k'] = value_lista

    df_demanda['fecha'] = df_demanda['fecha'].apply(lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.to_datetime(x, errors='coerce') is not pd.NaT else x)


    return df_demanda

df_demanda = demanda_datos(lang, category, widget, query) 

df_demanda


,fecha,energia_consumida /k
0,31/08/2023,691964.483
1,01/09/2023,702456.297
2,02/09/2023,611501.885
3,03/09/2023,561966.054
4,04/09/2023,675539.705
5,05/09/2023,696676.646
6,06/09/2023,708136.136
7,07/09/2023,713401.703
8,08/09/2023,691678.318
9,09/09/2023,618713.347


In [35]:
df_demanda.to_csv('demanda_evolucion.csv')